# Base

In [1]:
import os
from torch.nn.modules.activation import LeakyReLU
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import random
import json
from numpy import genfromtxt
import seaborn as sns

c:\Users\Edouard\github\Stress_Detetion_Pupil_Tracking\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DIR_NETWORK="/content/network/"
DIR_DATA="/content/data_treated/" #Path where you store your treated data

In [3]:
manualSeed=1

torch.manual_seed(manualSeed)
random.seed(manualSeed)
np.random.seed(manualSeed)
g = torch.Generator()
g.manual_seed(manualSeed)


In [ ]:
# l= os.listdir("/content/data")

In [ ]:
name_list=['data_ext_t2_experimental.txt.json', 'data_ext_t4_control.txt.json',
 'data_ext_t0_experimental.txt.json', 'data_ext_t1_control.txt.json',
 'data_ext_t1_experimental.txt.json', 'data_ext_t0_control.txt.json',
 'data_ext_t3_control.txt.json', 'data_ext_t2_control.txt.json',
 'data_ext_t4_experimental.txt.json', 'data_ext_t3_experimental.txt.json']

# Functions

In [4]:
def conf_mat(net,datal,trsh):
  x=datal[0].float().to(device)
  y=net(x).view(-1)
  pred=(y>trsh).int()
  label=datal[1].float().to(device).view(-1).int()
  comp=torch.eq(label,pred).int()
  mat_nolbl=np.zeros((2,2))
  for i in range(0,2):
    tens=torch.where(label==i,1,0)
    numtot=torch.sum(tens).item()
    num_G=torch.sum(torch.where(torch.mul(tens,comp)==1,1,0)).item()
    if i ==1:
      mat_nolbl[0,0]+=num_G
      mat_nolbl[1,0]+=numtot-num_G
    else:
      mat_nolbl[1,1]+=num_G
      mat_nolbl[0,1]+=numtot-num_G
  return mat_nolbl

# DS creation kfold

In [ ]:
class ds_pd(Dataset):
    def __init__(self, list_sample,mean_global,std_global):
        self.samples = []
        mean=mean_global
        std=std_global
        for i in range(0,len(list_sample)):
            label=list_sample[i][1]
            x=(np.array(list_sample[i][0])-mean)/(std)
            self.samples.append((x,label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, id):
        return self.samples[id]

In [ ]:
"""
Create the list that will be used to generate the datasets.
Our data files include 14 subject from control group and 17 subject from experimental group.
Experimental subject 11 will be the testing subject (out of cross validation).
As a cross validation is used in this project, the training datasets will be made from data of 13 subject of control group + 15 subject of experimental group
The validation datasets will be made of 1 subject from control group and 1 subject from experimental group.

The output is a list containing fold element. The cross validation with this configuration is a 195-fold crossvalidation.
A fold element is [data_for_training;data_for_validation;experimental_subject_used_for_validation_index;control_subject_used_for_validation_index]
"""
ds_list=[]
for v1 in range(0,16):
  if (v1!=11):
    for v2 in range(0,13):
      L_val=[]
      L_train=[]
      ind=0
      for name in name_list:
        f=open(DIR_DATA+name)
        group=name[12:-9]
        num=name[10:11]
        L = json.load(f)
        if group=="control":
          label=0
          for k in range(0,len(L)):
            if k==v2:
              for i in range(0,len(L[k])):
                L_val.append([L[k][i],label])
            else :
              for i in range(0,len(L[k])):
                L_train.append([L[k][i],label])
        if group=="experimental":
          if int(num)>=2:
            label=1
            ind+=1
          else:
            label=0
          for k in range(0,len(L)):
            if k==v1:
              for i in range(0,len(L[k])):
                L_val.append([L[k][i],label])
            else :
              if label==0 and k!=11:
                for i in range(0,len(L[k])):
                  L_train.append([L[k][i],label])
              if label==1 and k!=11:
                for i in range(0,len(L[k])):
                  L_train.append([L[k][i],label])
      ds_list.append([L_train,L_val,v1,v2])

In [ ]:
""" Generate the 195 datasets from the list for cross-validation"""

fold_list=[]
mean_t=[np.mean(a[0]) for a in ds_list[0][0]]
mean_v=[np.mean(a[0]) for a in ds_list[0][1]]
mean_global=np.mean(mean_t+mean_v)

std_t=[np.std(a[0]) for a in ds_list[0][0]]
std_v=[np.std(a[0]) for a in ds_list[0][1]]
std_global=np.mean(std_t+std_v)
for ds in ds_list:
  ds_t=ds_pd(ds[0],mean_global,std_global)
  ds_v=ds_pd(ds[1],mean_global,std_global)
  fold_list.append([ds_t,ds_v,ds[2],ds[3]])

# Model

In [ ]:
""" DNN Model """


def init_weight(m):
    """Initialization of the weights"""
    if isinstance(m,nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if isinstance(m, nn.BatchNorm1d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()


class Classifier_PD(nn.Module):
    """DNN model, see the model architecture in the report for more details"""
    def __init__(self, ngpu):
        super(Classifier_PD, self).__init__()
        self.ngpu = ngpu
        self.nnPD = nn.Sequential(
            nn.Linear(125,64,bias=True),
            nn.BatchNorm1d(64),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2),
            nn.Linear(64,32,bias=True),
            nn.BatchNorm1d(32),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2),
            nn.Linear(32,8,bias=True),
            nn.BatchNorm1d(8),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2),
            nn.Linear(8,1,bias=True),
            nn.Sigmoid()
        )
        self.nnPD.apply(init_weight)

    def forward(self, input):
        return self.nnPD(input)    

''

# Training

In [ ]:
def training(net,dataloader_t,dataloader_v,num_epochs,j,k):
  """ Training with a BCELoss on the dataset, for each epoch the net weights are saved and the mean error is computed to plot the loss
  for training and valdiation dataset
  The training is done using data augmentation with noise, a learning rate scheduler and checkpoints
  At each epoch network and optimizer data are saved in DIR_NETWORK repository
  """
  Loss = []
  Lossv= []
  lr=0.0001
  beta1=0.9
  optimizer = optim.Adam(net.parameters(), lr=lr, betas=(beta1, 0.999),weight_decay=0.005)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=15)
  temp_lr=lr
  temp_best_loss=100000
  for epoch in range(num_epochs):
      L_t=[]
      L_v=[]
      for i, dataj in enumerate(dataloader_t, 0):
          net.zero_grad()
          x=dataj[0].float().to(device)
          noise=np.sqrt(0.008)*torch.randn(x.shape).float().to("cuda")
          xnoise=x+noise
          yhat=dataj[1].float().to(device)
          yhat=yhat.view(-1,1)
          y=net(x)
          err_t=nn.BCELoss()(y.float(),yhat.float())
          err_t.backward()
          optimizer.step()
          L_t.append(err_t.item())
      for i, dataj in enumerate(dataloader_v, 0):
        net.eval()     
        x=dataj[0].float().to(device)
        yhat=dataj[1].float().to(device)
        yhat=yhat.view(-1,1)
        y=net(x)
        err_v=nn.BCELoss()(y.float(),yhat.float())
        L_v.append(err_v.item())
      err=np.mean(L_t)
      errv=np.mean(L_v)
      Loss.append(err)
      Lossv.append(errv)
      torch.save(net.state_dict(), DIR_NETWORK+"net_"+str(j)+"_"+str(k)+"/net_"+str(j)+"_"+str(k)+"_epoch_"+str(epoch)+".pth")
      torch.save(optimizer.state_dict(), DIR_NETWORK+"optim_"+str(j)+"_"+str(k)+"/optim_"+str(j)+"_"+str(k)+"_epoch_"+str(epoch)+".pth")
      if temp_best_loss>errv:
        temp_best_loss=errv
        temp_best_epoch=epoch
        #print("update best : {}".format(temp_best_epoch))
      scheduler.step(errv)
      if temp_lr>optimizer.state_dict()["param_groups"][0]["lr"]:
        temp_lr=optimizer.state_dict()["param_groups"][0]["lr"]
        net.load_state_dict(torch.load(DIR_NETWORK+"net_"+str(j)+"_"+str(k)+"/net_"+str(j)+"_"+str(k)+"_epoch_"+str(temp_best_epoch)+".pth"))
        dict_optim=torch.load(DIR_NETWORK+"optim_"+str(j)+"_"+str(k)+"/optim_"+str(j)+"_"+str(k)+"_epoch_"+str(temp_best_epoch)+".pth")
        dict_optim["param_groups"][0]["lr"]=temp_lr
        optimizer.load_state_dict(dict_optim)

  return [Loss,Lossv,np.argmin(Lossv)]

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

class NpEncoder(json.JSONEncoder):
  def default(self, obj):
      if isinstance(obj, np.integer):
          return int(obj)
      if isinstance(obj, np.floating):
          return float(obj)
      if isinstance(obj, np.ndarray):
          return obj.tolist()
      return super(NpEncoder, self).default(obj)


num_workers = 2
batch_size = 256
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

L=[]    
cntr=0
M=range(len(fold_list)//2,3*(len(fold_list)//4),1)
for iteration in tqdm(M):
  ds=fold_list[iteration]
  net= Classifier_PD(ngpu).to(device)
  dataset_t=ds[0]
  dataset_v=ds[1]
  k=ds[2]
  j=ds[3]
  os.mkdir(DIR_NETWORK+"net_{}_{}".format(j,k))
  os.mkdir(DIR_NETWORK+"optim_{}_{}".format(j,k))
  dataloader_t = torch.utils.data.DataLoader(dataset_t,batch_size=batch_size,shuffle=True,num_workers=num_workers,worker_init_fn=seed_worker,generator=g, drop_last=True)
  dataloader_v = torch.utils.data.DataLoader(dataset_v,batch_size=batch_size,shuffle=True,num_workers=num_workers,worker_init_fn=seed_worker,generator=g, drop_last=True)
  L=training(net,dataloader_t,dataloader_v,250,j,k)
  with open(DIR_NETWORK+"results_{}_{}.json".format(j,k),"w") as file:
    json.dump(L,file,cls=NpEncoder)
  

# Load and Results Analysis

In [ ]:
confusionmean=np.zeros((2,2))
acc_list=[]
prec_list=[]
recall_list=[]
f1_list=[]
confusion_list=[]
M=range(0,len(fold_list),1)
for n,m in zip(tqdm(M),range(0,len(M))):
  k=fold_list[n][2]
  j=fold_list[n][3]
  dataset_t=fold_list[n][0]
  dataset_v=fold_list[n][1]
  with open("/content/data/results/results_{}_{}.json".format(j,k),"r") as file:
    L=json.load(file)
  epoch=np.argmin(L[1])
  net= Classifier_PD(ngpu).to(device)
  net.load_state_dict(torch.load("/content/data/results/net_"+str(j)+"_"+str(k)+"/net_"+str(j)+"_"+str(k)+"_epoch_"+str(epoch)+".pth"))
  dataloader_t = torch.utils.data.DataLoader(dataset_t,batch_size=batch_size,shuffle=True,num_workers=num_workers,worker_init_fn=seed_worker,generator=g, drop_last=True)
  dataloader_v = torch.utils.data.DataLoader(dataset_v,batch_size=batch_size,shuffle=True,num_workers=num_workers,worker_init_fn=seed_worker,generator=g, drop_last=True)
  trsh=0.525
  net.eval()
  confusion=np.zeros((2,2))
  length_dsv=0
  for i, datal in enumerate(dataloader_v, 0):
        confusiont=conf_mat(net,datal,trsh)
        confusion+=confusiont
        length_dsv+=batch_size
  TP=confusion[0,0]
  TN=confusion[1,1]
  FN=confusion[1,0]
  FP=confusion[0,1]
  acc=(TP+TN)/(TP+FP+FN+TN)
  precision=TP/(TP+FP)
  recall=TP/(TP+FN)
  F1score=(2*recall*precision)/(recall+precision)
  acc_list.append(acc)
  prec_list.append(precision)
  recall_list.append(recall)
  f1_list.append(F1score)
  confusion_list.append(100*confusion/length_dsv)

confusionmean=np.round(np.mean(confusion_list,axis=0),3)
confusionstd=np.round(np.std(confusion_list,axis=0),3)
annot_confusion=np.array([str(a)+"+/-"+str(b) for a,b in zip(confusionmean.reshape(-1).tolist(),confusionstd.reshape(-1).tolist())]).reshape(confusionmean.shape)
x_axis_conf = ['stress','no stress']
y_axis_conf = ['stress','no stress']
sns.set(rc={"figure.figsize":(15, 5)})
fig, axs = plt.subplots(ncols=1,figsize=(15,9))
sns.heatmap(confusionmean.astype('int32'), xticklabels=x_axis_conf, yticklabels=y_axis_conf,annot=annot_confusion,ax=axs,fmt = '')
axs.set_xlabel('Ground Truth')
axs.set_ylabel('Prediction')
axs.title.set_text('Confusion Matrix label : Stress/No Stress (population in %)')

# Metrics

In [ ]:
mes=acc_list
plt.hist(mes,bins=20)
plt.title('Accuracy of all best models')
plt.xlabel('Accuracy') 
plt.ylabel('Number of models')
txtm="mean: " +str(round(np.mean(mes),3))
txtstd="std: " +str(round(np.std(mes),3))
plt.text(0.4, 20, txtm)
plt.text(0.4, 17, txtstd)
plt.show()

In [ ]:
mes=prec_list
plt.hist(mes,bins=20)
plt.title('Precision of all best models')
plt.xlabel('Precision') 
plt.ylabel('Number of models')
txtm="mean: " +str(round(np.mean(mes),3))
txtstd="std: " +str(round(np.std(mes),3))
plt.text(0.2, 18, txtm)
plt.text(0.2, 16, txtstd)
plt.show()

In [ ]:
mes=recall_list
plt.hist(mes,bins=20)
plt.title('Recall of all best models')
plt.xlabel('Recall') 
plt.ylabel('Number of models')
txtm="mean: " +str(round(np.mean(mes),3))
txtstd="std: " +str(round(np.std(mes),3))
plt.text(0.2, 35, txtm)
plt.text(0.2, 31, txtstd)
plt.show()

In [ ]:
mes=f1_list
plt.hist(mes,bins=20)
plt.title('F1 score of all best models')
plt.xlabel('F1 score') 
plt.ylabel('Number of models')
txtm="mean: " +str(round(np.mean(mes),3))
txtstd="std: " +str(round(np.std(mes),3))
plt.text(0.3, 22, txtm)
plt.text(0.3, 16, txtstd)
plt.show()